<a href="https://colab.research.google.com/github/rjrizani/pdf_scraper/blob/main/extract_pdf_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.1 MB/s eta 0:00:00


In [2]:
import pdfplumber
import pandas as pd

# Open the PDF file
pdf_path = "/content/Original.pdf"  # Replace with your actual file path
output_csv = "output.csv"

data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                data.append(row)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(data)
df.to_csv(output_csv, index=False, header=False)

print(f"Data extracted and saved to {output_csv}")

Data extracted and saved to output.csv


In [4]:
df.head()

,0
0,35-0844079 PAUL H ROHE COMPANY INC AURORA (812...
1,"35-0918397 RIETH-RILEY CONSTRUCTION CO., INC. ..."
2,35-1128289 BEATY CONSTRUCTION INC BOGGSTOWN (3...
3,35-1139301 E & B PAVING LLC ANDERSON (765)643-...
4,35-1151771 CALUMET CIVIL CONTRACTORS INC WHITE...


In [5]:
# Assuming your data is currently in a single column, let's say column 0
# Adjust the column index if your data is in a different column
column_with_data = 0

# Define a regular expression pattern to capture the different parts
# This pattern is an example and might need adjustment based on variations in your data
# Explanation of the pattern:
# (\S+): Capture one or more non-whitespace characters (for the code)
# \s+: One or more whitespace characters
# (.*?): Capture any character (except newline) zero or more times, non-greedily (for the company name)
# \s+: One or more whitespace characters
# (\w+): Capture one or more word characters (for the city/address)
# \s*: Zero or more whitespace characters
# (\(\d{3}\)\d{3}-\d{4}): Capture a phone number in the format (XXX)XXX-XXXX
# \s*: Zero or more whitespace characters
# (Yes|No): Capture either "Yes" or "No"
# (.*): Capture the rest of the string (for email and state code)

pattern = r'^(\S+)\s+(.*?)\s+(\w+)\s*(\(\d{3}\)\d{3}-\d{4})\s*(Yes|No)\s*(.*)$'

# Apply the pattern to the specified column and create new columns
extracted_data = df[column_with_data].str.extract(pattern, expand=True)

# Assign the extracted columns to new columns in the DataFrame
df['code'] = extracted_data[0]
df['placeholders'] = extracted_data[1]
df['address'] = extracted_data[2]
df['phone/fax'] = extracted_data[3]
df['valid for bid'] = extracted_data[4]
df['additional_data'] = extracted_data[5] # You might want to further process this

# Display the first few rows with the new columns
print("DataFrame with columns extracted using regex:")
display(df.head())

DataFrame with columns extracted using regex:


,0,code,placeholders,address,phone/fax,valid for bid,additional_data
0,35-0844079 PAUL H ROHE COMPANY INC AURORA (812...,35-0844079,PAUL H ROHE COMPANY INC,AURORA,(812)926-1471,Yes,mark.richardson@jrjnet.com IN
1,"35-0918397 RIETH-RILEY CONSTRUCTION CO., INC. ...",35-0918397,"RIETH-RILEY CONSTRUCTION CO., INC.",INDIANAPOLIS,(317)634-5561,Yes,lbender@rieth-riley.com IN (317)631-6265
2,35-1128289 BEATY CONSTRUCTION INC BOGGSTOWN (3...,35-1128289,BEATY CONSTRUCTION INC,BOGGSTOWN,(317)835-2254,Yes,bfenimore@beatyinc.com IN (317)835-2913
3,35-1139301 E & B PAVING LLC ANDERSON (765)643-...,35-1139301,E & B PAVING LLC,ANDERSON,(765)643-5358,Yes,quotes@ebpaving.com IN
4,35-1151771 CALUMET CIVIL CONTRACTORS INC WHITE...,35-1151771,CALUMET CIVIL CONTRACTORS INC,WHITESTOWN,(317)727-3284,Yes,quotes@calumetcivil.com IN (317)769-7424


In [9]:
df['placeholders'] = df['placeholders'].astype(str) + ' ' + df['additional_data'].astype(str)

# Hapus kolom 'additional_data'
df = df.drop(columns=['additional_data'])

# Tampilkan beberapa baris pertama untuk memeriksa hasilnya
print("DataFrame setelah menggabungkan dan menghapus kolom:")
display(df.head())

DataFrame setelah menggabungkan dan menghapus kolom:


,code,placeholders,address,phone/fax,valid for bid
0,35-0844079,PAUL H ROHE COMPANY INC mark.richardson@jrjnet...,AURORA,(812)926-1471,Yes
1,35-0918397,"RIETH-RILEY CONSTRUCTION CO., INC. lbender@rie...",INDIANAPOLIS,(317)634-5561,Yes
2,35-1128289,BEATY CONSTRUCTION INC bfenimore@beatyinc.com ...,BOGGSTOWN,(317)835-2254,Yes
3,35-1139301,E & B PAVING LLC quotes@ebpaving.com IN,ANDERSON,(765)643-5358,Yes
4,35-1151771,CALUMET CIVIL CONTRACTORS INC quotes@calumetci...,WHITESTOWN,(317)727-3284,Yes


In [10]:

output_csv_file = "output_modified.csv"

df.to_csv(output_csv_file, index=False)

print(f"DataFrame berhasil disimpan ke {output_csv_file}")


DataFrame berhasil disimpan ke output_modified.csv
